In [1]:
import reflex as rx
from sqlmodel import create_engine, Session, select
from weightsapp import LoggedExercise

In [2]:
sqlite_fn = "/home/mattc/code/logit/data.db"
sqlite_url = f"sqlite:///{sqlite_fn}"

engine = create_engine(sqlite_url, echo=True)


In [3]:
with Session(engine) as session:
    statement = select(LoggedExercise)
    results = session.exec(statement)
    for r in results:
        print(r)

2023-09-19 14:56:09,285 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-19 14:56:09,287 INFO sqlalchemy.engine.Engine SELECT loggedexercise.id, loggedexercise.created_datetime, loggedexercise.ename, loggedexercise.enum, loggedexercise.reps, loggedexercise.weight 
FROM loggedexercise
2023-09-19 14:56:09,287 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()
ename='Scan' enum=2 weight=70.0 reps=5 created_datetime='19-09-23' id=3
ename='Scan' enum=1 weight=70.0 reps=5 created_datetime='19-09-23' id=4
2023-09-19 14:56:09,288 INFO sqlalchemy.engine.Engine ROLLBACK


In [4]:
import pandas as pd
def _log_as_dataframe() -> pd.DataFrame:
    
    with Session(engine) as session:
        ex_list = (
            session.query(LoggedExercise).all()
        )
    ex_list = [ex._li() for ex in ex_list]    
    return pd.DataFrame(ex_list,
                            columns=["date", "ename", "enum", "reps", "kg"])

In [5]:
df = _log_as_dataframe()

2023-09-19 14:56:09,312 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-19 14:56:09,313 INFO sqlalchemy.engine.Engine SELECT loggedexercise.id AS loggedexercise_id, loggedexercise.created_datetime AS loggedexercise_created_datetime, loggedexercise.ename AS loggedexercise_ename, loggedexercise.enum AS loggedexercise_enum, loggedexercise.reps AS loggedexercise_reps, loggedexercise.weight AS loggedexercise_weight 
FROM loggedexercise
2023-09-19 14:56:09,314 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ()
2023-09-19 14:56:09,315 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
other_data = pd.DataFrame([
    ["20-09-23", "Run", "1", "10", "1000"],
    ["21-09-23", "Other", "2", "20", "500"],
    ["21-09-23", "Other", "2", "30", "500"],

                           ],
                             columns=df.columns)

df = pd.concat([df, other_data])

In [12]:
df

,date,ename,enum,reps,kg
0,19-09-23,Scan,2,5,70.0
1,19-09-23,Scan,1,5,70.0
0,20-09-23,Run,1,10,1000
1,21-09-23,Other,2,20,500


In [11]:
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure(
    data=[go.Line(x=df['date'], y=df['reps'])],
        layout=go.Layout(
        title=go.layout.Title(text="A Figure Specified By A Graph Object")
    )
)
fig.show()

/home/mattc/venvs/py39/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


